In [1]:
#Load in utility classes
%run ~/projects/testlipids/testlipids.ipynb
import os #Operating system specific commands
import re #Regular expression library

membrane="epithelial"
insane="./insane+SF.py"
mdparams="./test.mdp"
martinipath="./martini.ff"

# TODO:  3000 lipids
# TODO:  neuronal membrane
# TODO:  plasma membrane

In [2]:
# Cleaning up intermediate files from previous runs
!rm -f *#*
!rm -f *step*
!rm -f {membrane}.gro
!rm -f {membrane}.edr
!rm -f {membrane}.log
!rm -f {membrane}.pdb
!rm -f {membrane}.tpr
!rm -f {membrane}.trr

In [3]:
cmdline = "-u PBSM:0.007 -u PXSM:0.004 -u DPSM:0.017 -u PNSM:0.008 -u DPMC:0.007 -u POMC:0.043 -u DOMC:0.009 -u PIMC:0.009 -u OIMC:0.004 -u OEMC:0.005 -u PAMC:0.012 -u PUMC:0.029 -u POME:0.001 -u DOME:0.006 -u OIME:0.001 -u OQME:0.002 -u OAME:0.014 -u OUME:0.006 -u IAME:0.007 -u IQME:0.001 -u LPPC:0.008 -u DPPC:0.003 -u POPC:0.147 -u DOPC:0.011 -u PIPC:0.088 -u OIPC:0.05 -u PAPC:0.052 -u PUPC:0.05 -u POPE:0.007 -u OPPE:0.002 -u DOPE:0.007 -u PAPE:0.006 -u PUPE:0.002 -u OIPE:0.002 -u PQPE:0.002 -u OQPE:0.001 -u OAPE:0.002 -u OUPE:0.001 -u POPG:0.004 -u DOPG:0.003 -u DODG:0.004 -u PODG:0.007 -l OGPS:0.004 -l DPPS:0.001 -l POPS:0.097 -l DOPS:0.006 -l PAPI:0.029 -l PAP1:0.014 -l PAP2:0.014 -l PQPI:0.02 -l POPI:0.013 -l PUPI:0.008 -l DOPI:0.008 -l PIPI:0.006 -l PEPI:0.004 -l PBSM:0.003 -l PXSM:0.002 -l DPSM:0.007 -l PNSM:0.003 -l DPMC:0.003 -l POMC:0.017 -l DOMC:0.004 -l PIMC:0.003 -l OIMC:0.002 -l OEMC:0.002 -l PAMC:0.005 -l PUMC:0.012 -l POME:0.005 -l DOME:0.021 -l OIME:0.004 -l OQME:0.006 -l OAME:0.054 -l OUME:0.024 -l IAME:0.025 -l IQME:0.003 -l LPPC:0.003 -l DPPC:0.001 -l POPC:0.058 -l DOPC:0.004 -l PIPC:0.035 -l OIPC:0.02 -l PAPC:0.021 -l PUPC:0.02 -l POPA:0.01 -l DOPA:0.005 -l POPE:0.025 -l OPPE:0.009 -l DOPE:0.027 -l PAPE:0.021 -l PUPE:0.008 -l OIPE:0.008 -l PQPE:0.007 -l OQPE:0.004 -l OAPE:0.009 -l OUPE:0.005 -l POPG:0.004 -l DOPG:0.002 -l DODG:0.002 -l PODG:0.004  "
 

In [5]:
print("Build")
build = !python2 {insane} -o {membrane}.gro -p {membrane}.top -d 0 -x 35 -y 35 -z 15 -sol PW -salt 0.15 -center -charge 0 -orient {cmdline}  
for line in build:
    print(line)


Build
; X: 35.000 (45 lipids) Y: 35.000 (45 lipids)
; 2025 lipids in upper leaflet, 2025 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 49974
; Charge of membrane: -317.000000
; Total charge: -317.000000
; NDX Solvent 49975 391151
; NDX System 1 391151


In [18]:
# remove extra descriptive lines in the gro file that would otherwise overrun the buffer during GROMPP
import datetime
generationDate = datetime.datetime.now().strftime("%Y.%m.%d")
!sed -i '1s/.*/Complex Epithelial membrane generated ({generationDate}) using INSANE/' {membrane}.gro

# remove extra descriptive lines in the top file that would otherwise overrun the buffer during GROMPP
import datetime
generationDate = datetime.datetime.now().strftime("%Y.%m.%d")
!sed -i '13s/.*/Complex Epithelial membrane generated ({generationDate}) using INSANE/' {membrane}.top

In [19]:

print("Grompp")
grompp = !gmx grompp -f {mdparams} -c {membrane}.gro -p {membrane}.top -o  {membrane}.tpr
success=True
for line in grompp:
    if re.search("Fatal error", line):
        success=False
    #if not success:
    print(line)
    
if success:
    print("Run")
    !export GMX_MAXCONSTRWARN=-1
    !export GMX_SUPPRESS_DUMP=1
    run = !gmx mdrun -v -deffnm {membrane}
    summary=""
    logfile = membrane+".log"
    if not os.path.exists(logfile):
        print("no log file")
        print("== === ====")
        for line in run:
            print(line)
    else:
        try:
            file = open(logfile, "r")
            fe = False
            for line in file:
                if fe:
                    success=False
                    summary=line
                elif re.search("^Steepest Descents.*converge", line):
                    success=True
                    summary=line
                    break
                elif re.search("Fatal error", line):
                    fe = True
        except IOError as exc:
            sucess=False;
            summary=exc;
    if success:
        print("Success")
    else:
        print(summary)


Grompp
                      :-) GROMACS - gmx grompp, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GRO

In [8]:
# test lipids
#Load in utility classes
%run ./testlipids.ipynb

#load lipids from GRO file
lipids=lipidsfromsystem("./epithelial")

print(lipids)

testlipids(lipids,membrane,insane,mdparams,martinipath).execute(True).report()

Converting ./epithelial.gro to a pdb file
using system specified in ./epithelial.pdb file
['PBSM', 'PXSM', 'DPSM', 'PNSM', 'DPMC', 'POMC', 'DOMC', 'PIMC', 'OIMC', 'OEMC', 'PAMC', 'PUMC', 'POME', 'DOME', 'OIME', 'OQME', 'OAME', 'OUME', 'IAME', 'IQME', 'LPPC', 'DPPC', 'POPC', 'DOPC', 'PIPC', 'OIPC', 'PAPC', 'PUPC', 'POPE', 'OPPE', 'DOPE', 'PAPE', 'PUPE', 'OIPE', 'PQPE', 'OQPE', 'OAPE', 'OUPE', 'POPG', 'DOPG', 'DODG', 'PODG', 'OGPS', 'DPPS', 'POPS', 'DOPS', 'PAPI', 'PAP1', 'PAP2', 'PQPI', 'POPI', 'PUPI', 'DOPI', 'PIPI', 'PEPI', 'POPA', 'DOPA']
[57]testing lipid PBSM in membrane epithelial...create...compile...minimize=[success]: 16.21 secs
[56]testing lipid PXSM in membrane epithelial...create...compile...minimize=[success]: 25.50 secs
[55]testing lipid DPSM in membrane epithelial...create...compile...minimize=[success]: 16.50 secs
[54]testing lipid PNSM in membrane epithelial...create...compile...minimize=[success]: 18.17 secs
[53]testing lipid DPMC in membrane epithelial...create...comp

In [9]:
# test lipids
#Load in utility classes
%run ./testlipids.ipynb

# test an individual lipis species
import os #Operating system specific commands
#re-compile
lipid = 'DPMC'
results=[]

if build(os.getcwd(),membrane,lipid,insane).execute().report(results).success:
    comp = compile(os.getcwd(),membrane,lipid,mdparams,martinipath).execute().report(results)
    print(lipid)
    if not comp.success:
        for line in comp.output:
            print(line)
else:
    for line in results:
        print(line)
print("done")

cp: './martini.ff' and '/home/richard/projects/epithelial/martini.ff' are the same file
DPMC
done
